# 导入依赖库

In [1]:
import os
import pandas as pd
import akshare as ak
from utils import get_etf_data

# 超参数

In [2]:
max_price = 50  # 价格要求
min_sample_size = 252  # 样本量要求
min_transaction_amount = 1e8 # 成交额要求
max_cor = 0.9
update = True
latest_trade_date = '2025-08-08'  # 最新交易日

# 工具函数

In [3]:
def get_etfs(update=False):
    """获取 ETF 实时行情"""
    filename = f"data/etf.csv"

    if not update and os.path.exists(filename):
        df = pd.read_csv(filename)

    else:
        df = ak.fund_etf_category_sina(symbol="ETF基金")
        df = df.sort_values(by='成交额', ascending=False)
        df.to_csv(filename, index=False)
        print(f'下载数据并存储到{filename}')

    return df

In [4]:
def deduplicate_etf_pool(etfs: pd.DataFrame,
                          window: int,
                          corr_threshold: float) -> pd.DataFrame:
    """
    对ETF池去重, 计算最近 window 天收益率相关性，
    对相关性高于阈值的ETF对, 仅保留第一次出现的ETF, 所以在去重前需根据关心的指标先进行排序。

    参数：
    - etfs: 包含'代码'列, 已按某指标从高到低排序的DataFrame
    - window: 计算相关性所需的样本天数
    - corr_threshold: 相关性阈值

    返回：
    - 过滤去重后的etfs子集
    """
    # 获取回溯日期范围
    benchmark = get_etf_data('510300')
    start_date = benchmark.index[-window]
    
    # 取收益率
    returns = {}
    for code in etfs['代码']:
        df = get_etf_data(code, start=start_date)
        returns[code] = df['涨跌幅']

    # 构建所有ETF的收益率DataFrame
    returns_df = pd.DataFrame(returns)

    # 计算相关性矩阵
    corr_mat = returns_df.corr()

    # 标记需要剔除的代码
    to_drop = set()
    codes = list(etfs['代码'])
    for i, code_i in enumerate(codes):
        for code_j in codes[i+1:]:
            if corr_mat.loc[code_i, code_j] > corr_threshold:
                # 保留先出现（成交额更大）的code_i，剔除code_j
                to_drop.add(code_j)

    # 返回过滤后的数据
    filtered = etfs[~etfs['代码'].isin(to_drop)].copy()
    return filtered

# 筛选股票池

In [5]:
# 价格与流动性初筛
primary_etfs = get_etfs(update=update)
primary_etfs = primary_etfs.sort_values(by='成交额', ascending=False)
primary_etfs = primary_etfs[(primary_etfs['最新价'] < max_price) & (primary_etfs['成交额'] > min_transaction_amount / 2)]
primary_etfs = primary_etfs[['代码', '名称']] 

下载数据并存储到data/etf.csv


In [6]:
# 样本量与流动性筛选
qualified_etfs = []
for i in range(len(primary_etfs)):
    code = primary_etfs.iat[i, 0][2:]
    df = get_etf_data(code, update=update, latest_trade_date=latest_trade_date)
    if df is not None:
        etf_dict = {
            '代码': code, 
            '名称': primary_etfs.iat[i, 1],
            '样本量': len(df),
            '日均成交额': df['成交额'].tail(min_sample_size).mean(),
            '日均涨跌幅': df['涨跌幅'].tail(min_sample_size).mean(),
        }
        qualified_etfs.append(etf_dict)
qualified_etfs = pd.DataFrame(qualified_etfs)
qualified_etfs = qualified_etfs[
    (qualified_etfs['日均成交额'] > min_transaction_amount) &  # 日均成交额要求
    (qualified_etfs['样本量'] > min_sample_size)
]   
qualified_etfs = qualified_etfs.sort_values(by='日均涨跌幅', ascending=False)
qualified_etfs.to_csv('data/qualified_etfs.csv', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]

下载数据并存储到data/etf/513090.csv
下载数据并存储到data/etf/511380.csv
下载数据并存储到data/etf/513120.csv
下载数据并存储到data/etf/512050.csv
下载数据并存储到data/etf/563360.csv
下载数据并存储到data/etf/159352.csv
下载数据并存储到data/etf/588000.csv
下载数据并存储到data/etf/159570.csv
下载数据并存储到data/etf/513130.csv
下载数据并存储到data/etf/513180.csv
下载数据并存储到data/etf/159351.csv
下载数据并存储到data/etf/159338.csv
下载数据并存储到data/etf/518880.csv
下载数据并存储到data/etf/513330.csv
下载数据并存储到data/etf/588200.csv
下载数据并存储到data/etf/511180.csv
下载数据并存储到data/etf/159361.csv
下载数据并存储到data/etf/159567.csv
下载数据并存储到data/etf/513060.csv
下载数据并存储到data/etf/510300.csv
下载数据并存储到data/etf/159915.csv
下载数据并存储到data/etf/512880.csv
下载数据并存储到data/etf/159792.csv
下载数据并存储到data/etf/512480.csv
下载数据并存储到data/etf/513050.csv
下载数据并存储到data/etf/159740.csv
下载数据并存储到data/etf/563800.csv
下载数据并存储到data/etf/159892.csv
下载数据并存储到data/etf/520500.csv
下载数据并存储到data/etf/159509.csv
下载数据并存储到data/etf/513750.csv
下载数据并存储到data/etf/159934.csv
下载数据并存储到data/etf/159217.csv
下载数据并存储到data/etf/562500.csv
下载数据并存储到data/etf/510050.csv
下载数据并存储到data/etf/512

In [7]:
# 相关性去重
deduplicate_etfs = deduplicate_etf_pool(qualified_etfs, window=min_sample_size, corr_threshold=max_cor)
deduplicate_etfs.to_csv('data/deduplicate_etfs.csv', index=False)